In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from TickersList import tickers
import re

In [2]:
# Column names that are needed in the pandas dataframe
column_names = ['Ticker','Company Name','BusinessType','Date','Open','High','Low','Beta',
                'VWAP','Market Cap All Classes', 'Dividend',
                'Div Freq', 'P/E Ratio', 'EPS','Yield',
                'P/B Ratio','Exchange']

In [3]:
# Create dictionary and fill with empty lists
tickerDict = {}
for i in column_names:
    tickerDict[i]=[None]*len(tickers)

In [4]:
# Extract the data from web for each ticker
for i in range(len(tickerDict['Ticker'])):
    tickerDict['Ticker'][i]=tickers[i]
    results = requests.get('https://web.tmxmoney.com/quote.php?qm_symbol=' + tickers[i])
    src = results.content
    soup = BeautifulSoup(src, 'lxml')
    texts = soup.find_all(text=True)
    # Get company name
    compName = soup.find_all('div', class_="quote-company-name")
    tickerDict['Company Name'][i] = compName[0].find('h4').text
    # Get the date
    dateTick = compName[0].find('p').text
    dateTick = dateTick.split('|')
    tickerDict['Date'][i] = re.sub('\s', '', dateTick[1])
    # most of the info is in the dq_card div class
    dq_card = soup.find_all('div', class_="dq-card")
    for dq in range(len(dq_card)):
        if 'Open' in dq_card[dq].text:
            tickerDict['Open'][i] = dq_card[dq].find('strong').string
        if 'High' in dq_card[dq].text:
            tickerDict['High'][i] = dq_card[dq].find('strong').string
        if 'Low' in dq_card[dq].text:
            tickerDict['Low'][i] = dq_card[dq].find('strong').string
        if 'Beta' in dq_card[dq].text:
            tickerDict['Beta'][i] = dq_card[dq].find('strong').string
        if 'VWAP' in dq_card[dq].text:
            tickerDict['VWAP'][i] = dq_card[dq].find('strong').string
        if 'All Classes' in dq_card[dq].text:
            tickerDict['Market Cap All Classes'][i] = dq_card[dq].find('strong').string
        if 'Dividend' in dq_card[dq].text:
            dividend = dq_card[dq].find('strong').string
            dividend = dividend.replace("\xa0", " ")
            tickerDict['Dividend'][i] = dividend
        if 'Div. Frequency' in dq_card[dq].text:
            tickerDict['Div Freq'][i] = dq_card[dq].find('strong').string
        if 'P/E Ratio' in dq_card[dq].text:
            tickerDict['P/E Ratio'][i] = dq_card[dq].find('strong').string
        if 'EPS' in dq_card[dq].text:
            tickerDict['EPS'][i] = dq_card[dq].find('strong').string
        if 'Yield' in dq_card[dq].text:
            tickerDict['Yield'][i] = dq_card[dq].find('strong').string
        if 'P/B Ratio' in dq_card[dq].text:
            tickerDict['P/B Ratio'][i] = dq_card[dq].find('strong').string
        if 'Exchange' in dq_card[dq].text:
            t = dq_card[dq].find('strong').string
            tickerDict['Exchange'][i] = re.sub('\s','',t)

In [5]:
df_td = pd.DataFrame(tickerDict, columns = column_names)

In [6]:
df_td

,Ticker,Company Name,BusinessType,Date,Open,High,Low,Beta,VWAP,Market Cap All Classes,Dividend,Div Freq,P/E Ratio,EPS,Yield,P/B Ratio,Exchange
0,BAM.A,Brookfield Asset Management Inc. Class A Limit...,None,"Mar30,2020,9:53AMEDT",59.61,59.88,58.17,1.13,59.10,"61,722,542,145",0.18 USD,Quarterly,17.00,3.49,1.611,1.385,TSX
1,BCE,BCE Inc.,None,"Mar30,2020,9:53AMEDT",54.57,54.80,53.65,0.4313,54.196,"48,770,394,371",0.8325 CAD,Quarterly,15.90,3.37,6.203,2.856,TSX
2,BIP.UN,Brookfield Infrastructure Partners L.P.,None,"Mar30,2020,9:53AMEDT",51.21,53.17,51.20,1.0151,51.723,"15,606,911,143",0.5375 USD,Quarterly,575.40,0.09,5.582,1.799,TSX
3,BNS,Bank of Nova Scotia (The),None,"Mar30,2020,9:53AMEDT",54.18,54.55,52.93,1.0707,54.106,"64,751,203,480",0.90 CAD,Quarterly,7.90,6.87,6.631,1.02,TSX
4,BPY.UN,Brookfield Property Partners L.P.,None,"Mar30,2020,9:53AMEDT",12.30,12.43,11.42,1.4774,11.929,"10,989,621,345",0.3325 USD,Quarterly,5.00,2.46,14.402,0.263,TSX
5,CAR.UN,Canadian Apartment Properties Real Estate Inve...,None,"Mar30,2020,9:53AMEDT",42.08,42.09,40.64,0.7977,41.364,"7,105,628,511",0.115 CAD,Monthly,5.60,7.53,3.276,0.846,TSX
6,CHE.UN,Chemtrade Logistics Income Fund,None,"Mar30,2020,9:53AMEDT",4.55,4.60,4.35,1.5735,4.5126,"423,186,853",0.05 CAD,Monthly,N/A,-1.08,12.632,0.557,TSX
7,ENB,Enbridge Inc.,None,"Mar30,2020,9:53AMEDT",38.89,39.50,38.13,1.0441,38.775,"77,317,763,817",0.81 CAD,Quarterly,14.70,2.64,8.335,1.326,TSX
8,FTS,Fortis Inc.,None,"Mar30,2020,9:53AMEDT",50.28,51.05,49.87,0.3227,50.372,"23,383,575,000",0.4775 CAD,Quarterly,13.10,3.79,3.843,1.383,TSX
9,GRT.UN,Granite Real Estate Investment Trust,None,"Mar30,2020,9:53AMEDT",55.00,55.04,53.65,0.7574,53.79,"2,922,062,581",0.242 CAD,Monthly,7.80,7.05,5.256,0.929,TSX
